<a href="https://colab.research.google.com/github/wildtulipan/Acetycholine-Striatal-Dynamics/blob/main/FiberPhotometry_Acetylcholine_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Acetylcholine Photometry

## Set up
Run this section before running everythin else

In [72]:
#@title Load Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from google.colab import drive
from scipy.signal import butter, filtfilt
from scipy.stats import linregress
from tqdm import tqdm



In [73]:
#@title Helper Functions
def parse_photometry_data(path):
    """
    Parses fiber photometry data from a directory structure and organizes it into a nested dictionary.
    The function expects folders with names containing animal ID, cell type, experiment, and session information
    in a specific format (10 underscore-separated parts). Each folder should contain:
    - Events.csv
    - Fluorescence-unaligned.csv
    - Fluorescence.csv
    - Outputs.csv

    Args:
        path (str): Path to the root directory containing photometry data folders

    Returns:
        dict: {animal_id: {'cell_type': str, experiment: {session: {data_files...}}}
                      where data_files includes all CSV contents as DataFrames
    Raises:
        SystemExit: If the specified directory doesn't exist
    """

    photometry_data_dict = {}
    photomerty_path = os.path.join(path, 'Fiber_Photometry')
    behavior_path = os.path.join(path, 'Behavior')

    # Check if path exists
    if not os.path.exists(path):
        print("Error: Directory doesn't exist")
        exit()

    print("\nFound folders:")
    print("ID\tCell\tExperiment\tSession")  # \t makes tab spacing
    print("-" * 40)

    # Go through each item in the folder
    for folder in os.listdir(photomerty_path):
        full_path = os.path.join(photomerty_path, folder)

        # Only process if it's a folder
        if os.path.isdir(full_path):
            parts = folder.split('_')

            # Check if folder name has enough parts
            if len(parts) == 10:
                # Get the parts we care about
                animal_id = int(parts[6])
                cell_type = str(parts[7])
                experiment = str(parts[8])
                session = int(parts[9])

                # Print the info with tab spacing
                print(f"{animal_id}\t{cell_type}\t{experiment}\t\t{session}")

                # Extract the files
                Events = pd.read_csv(os.path.join(full_path, 'Events.csv'))
                Fluorescence_unaligned = pd.read_csv(os.path.join(full_path, 'Fluorescence-unaligned.csv'))
                Fluorescence = pd.read_csv(os.path.join(full_path, 'Fluorescence.csv'))
                Outputs = pd.read_csv(os.path.join(full_path, 'Outputs.csv'))

                # Set up the data format
                if animal_id not in photometry_data_dict:
                    photometry_data_dict[animal_id] = {}
                    photometry_data_dict[animal_id]['cell_type'] = cell_type
                if experiment not in photometry_data_dict[animal_id]:
                    photometry_data_dict[animal_id][experiment] = {}
                if session not in photometry_data_dict[animal_id][experiment]:
                    photometry_data_dict[animal_id][experiment][session] = {
                        'Events': Events,
                        'Fluorescence-unaligned': Fluorescence_unaligned,
                        'Fluorescence': Fluorescence,
                        'Outputs': Outputs
                    }

    return photometry_data_dict


def bandpass_filter_signal(signal, lowcut, highcut, fs, order=4):
    """
    Apply a zero-phase Butterworth bandpass filter to a signal.

    Args:
        signal (np.ndarray): Input signal (1D array)
        lowcut (float): Lower cutoff frequency (Hz)
        highcut (float): Upper cutoff frequency (Hz)
        fs (float): Sampling rate (Hz)
        order (int): Filter order

    Returns:
        np.ndarray: Filtered signal
    """
    # Normalize frequencies by Nyquist (fs/2)
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist

    # Design Butterworth bandpass filter
    b, a = butter(order, [low, high], btype='band')

    # Apply zero-phase filtering (filtfilt avoids phase distortion)
    filtered_signal = filtfilt(b, a, signal)

    return filtered_signal

def process_photometry_data(photometry_data_dict, lowcut=0.3, highcut=5, fs=15):
    """
    Processes photometry data by:
    1. Aligning 470nm and 410nm signals
    2. Applying bandpass filtering
    3. Performing motion correction via linear regression
    4. Storing all results in the input dictionary

    Args:
        photometry_data_dict (dict): Nested photometry data structure
        lowcut (float): Low cutoff frequency for bandpass filter
        highcut (float): High cutoff frequency for bandpass filter
        fs (float): Sampling rate in Hz

    Returns:
        dict: Modified photometry_data_dict with added processed data
    """

    # Initialize progress bar (optional)
    total_sessions = sum(len(exp) for animal in photometry_data_dict.values()
                        for exp in animal.values() if isinstance(exp, dict))

    with tqdm(total=total_sessions, desc="Processing sessions") as pbar:
        for animal_id, animal_data in photometry_data_dict.items():
            cell_type = animal_data['cell_type']

            for experiment_name, sessions in animal_data.items():
                if experiment_name == 'cell_type':
                    continue

                for session_num, session_data in sessions.items():
                    # Update progress bar
                    pbar.set_postfix({
                        'Animal': animal_id,
                        'Experiment': experiment_name,
                        'Session': session_num
                    })
                    pbar.update(1)

                    # Get raw fluorescence data
                    fluo_unaligned = session_data['Fluorescence-unaligned']

                    try:
                        # Extract and align signals
                        fluorescence_470 = np.array(fluo_unaligned['Channel1'][fluo_unaligned['Lights'] == 470])
                        fluorescence_410 = np.array(fluo_unaligned['Channel1'][fluo_unaligned['Lights'] == 410])
                        min_length = min(len(fluorescence_470), len(fluorescence_410))
                        fluorescence_470 = fluorescence_470[:min_length]
                        fluorescence_410 = fluorescence_410[:min_length]

                        # Filter signals
                        fluorescence_470_filt = bandpass_filter_signal(
                            fluorescence_470, lowcut=lowcut, highcut=highcut, fs=fs
                        )
                        fluorescence_410_filt = bandpass_filter_signal(
                            fluorescence_410, lowcut=lowcut, highcut=highcut, fs=fs
                        )

                        # Motion correction
                        slope, intercept, r_value, p_value, std_err = linregress(
                            x=fluorescence_410_filt,
                            y=fluorescence_470_filt
                        )
                        fluorescence_470_est_motion = intercept + slope * fluorescence_410_filt
                        fluorescence_470_corrected = fluorescence_470_filt - fluorescence_470_est_motion

                        # Store all processed data
                        session_data['processed'] = {
                            'corrected': fluorescence_470_corrected,
                            'est_motion': fluorescence_470_est_motion,
                            '470_filt': fluorescence_470_filt,
                            '410_filt': fluorescence_410_filt,
                            '470_raw': fluorescence_470,
                            '410_raw': fluorescence_410,
                            'regression_params': {
                                'slope': slope,
                                'intercept': intercept,
                                'r_value': r_value,
                                'p_value': p_value,
                                'std_err': std_err
                            }
                        }

                    except Exception as e:
                        print(f"Error processing {animal_id}/{experiment_name}/{session_num}: {str(e)}")
                        continue

    return photometry_data_dict

In [74]:
# @title Mount Google Drive
drive.mount('/content/drive')
notebook_path = '/content/drive/MyDrive/Research/Striatum_Acetylcholine'
print("Notebook is in:", notebook_path)
os.chdir(notebook_path) # Set it as the working directory
print("Working directory set to notebook's location.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Notebook is in: /content/drive/MyDrive/Research/Striatum_Acetylcholine
Working directory set to notebook's location.


In [75]:
#@title Data Loader
path = "/content/drive/MyDrive/Research/Striatum_Acetylcholine/Data"
photometry_data_dict = parse_photometry_data(path)
photometry_data_dict = process_photometry_data(photometry_data_dict)


Found folders:
ID	Cell	Experiment	Session
----------------------------------------
8620	A2a	UR		1
8620	A2a	AQ		1
8620	A2a	AQ		9
8620	A2a	AQ		10
8620	A2a	UP		1
8620	A2a	UP		3
8620	A2a	UP		2


Processing sessions: 100%|██████████| 7/7 [00:00<00:00, 138.35it/s, Animal=8620, Experiment=UP, Session=2]


## Analysis

In [76]:
#@title Preprocesing Figures

# This cell generate figures to better